# Welcome  

Notebook Author: Samuel Alter  
Notebook Subject: Capstone Project - Geographic Analysis

BrainStation Winter 2023: Data Science

## Initial Setup

### Imports

In [50]:
import pandas as pd
import numpy as np

# import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [31]:
# import geographic information .CSV

sm_geo_original=pd.read_csv('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/joins/sm_geo_combine.csv')
sm_geo_original

,objectid,perim_id,perim_elevation_1,perim_asp_1,fire,geometry
0,4751.0,1644.0,434.0,31.883564,1,MULTIPOLYGON (((-118.297645979672 34.135057069...
1,4751.0,1645.0,349.0,7.624194,1,MULTIPOLYGON (((-118.297645979672 34.135057069...
2,4751.0,1646.0,288.0,72.613029,1,MULTIPOLYGON (((-118.297645979672 34.135057069...
3,4751.0,1647.0,189.0,335.170654,1,MULTIPOLYGON (((-118.297645979672 34.135057069...
4,4751.0,1648.0,229.0,346.464142,1,MULTIPOLYGON (((-118.297645979672 34.135057069...
...,...,...,...,...,...,...
19140,0.0,5935.0,52.0,149.036240,0,POINT (-118.2930000000007 34)
19141,0.0,5936.0,51.0,185.194427,0,POINT (-118.2880000000007 34)
19142,0.0,5937.0,52.0,270.000000,0,POINT (-118.2830000000007 34)
19143,0.0,5938.0,54.0,341.565063,0,POINT (-118.2780000000007 34)


### EDA

In [32]:
sm_geo_original.isna().sum()

objectid               0
perim_id             641
perim_elevation_1    641
perim_asp_1          668
fire                   0
geometry               0
dtype: int64

In [33]:
sm_geo_original[sm_geo_original['perim_asp_1'].isna()==True]

,objectid,perim_id,perim_elevation_1,perim_asp_1,fire,geometry
14,4753.0,NaN,NaN,NaN,1,MULTIPOLYGON (((-118.305363835672 34.119507991...
15,4767.0,NaN,NaN,NaN,1,MULTIPOLYGON (((-118.843533279074 34.169495387...
24,4796.0,NaN,NaN,NaN,1,MULTIPOLYGON (((-118.60574361788 34.1457095562...
25,4849.0,NaN,NaN,NaN,1,MULTIPOLYGON (((-118.377669263434 34.122895200...
26,4850.0,NaN,NaN,NaN,1,MULTIPOLYGON (((-118.574062124596 34.079625547...
...,...,...,...,...,...,...
18743,0.0,4600.0,32.0,NaN,0,POINT (-118.3680000000007 34.04)
18786,0.0,4752.0,53.0,NaN,0,POINT (-118.4330000000006 34.035)
18810,0.0,4776.0,67.0,NaN,0,POINT (-118.3130000000007 34.035)
18863,0.0,4941.0,52.0,NaN,0,POINT (-118.3130000000007 34.03)


In [34]:
(sm_geo_original.isna().sum())/(sm_geo_original.count())*100

objectid             0.000000
perim_id             3.464116
perim_elevation_1    3.464116
perim_asp_1          3.615306
fire                 0.000000
geometry             0.000000
dtype: float64

### Prepare data for analysis

I have enough data that $3.5{\%}$ (roughly $650$ rows) is not a large portion of the dataset. I will remove these `NaN` rows:

In [36]:
sm_geo_original=sm_geo_original.dropna()
sm_geo_original.isna().sum()

objectid             0
perim_id             0
perim_elevation_1    0
perim_asp_1          0
fire                 0
geometry             0
dtype: int64

In [37]:
X=sm_geo_original[['perim_elevation_1','perim_asp_1']]
y=sm_geo_original[['fire']]

y

,fire
0,1
1,1
2,1
3,1
4,1
...,...
19140,0
19141,0
19142,0
19143,0


In [38]:
print(X.shape)
print(y.shape)

(18477, 2)
(18477, 1)


In [52]:
y=ravel(y)

NameError: name 'ravel' is not defined

In [47]:
y.sum()/y.count()

fire    0.889809
dtype: float64

Bare minimum

## Modeling

### Train, Test, Split

Partition dataset to have $0.\overline3$ of the total as testing.

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(1/3),stratify=y)

In [51]:
# instantiate model
logreg=LogisticRegression()

# fit the model
logreg.fit(X_train,y_train)

print('Training Score: ',logreg.score(X_train,y_train))
print('Testing Score:  ',logreg.score(X_test,y_test))

Training Score:  0.8898360123396656
Testing Score:   0.889754830329599


/Applications/Anaconda/anaconda3/envs/sklearn/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
